In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

hours = pd.read_excel('Data/Consulting Hours.xlsx')
capital = pd.read_excel('Data/Capital Milestones.xlsx')
established = pd.read_excel('Data/Business Established.xlsx')

In [3]:
hours = hours.dropna(subset='Client ID')
capital = capital.dropna(subset='Client ID')
established = established.dropna(subset='Client ID')

In [4]:
import pandasql as pds

query = """
SELECT hours.*, capital.*
FROM hours
LEFT JOIN capital
ON hours."Client ID" = capital."Client ID"
ORDER BY hours."Session Date"
"""

bus_capital = pds.sqldf(query, locals())
bus_capital['Session Date'] = pd.to_datetime(bus_capital['Session Date'])
bus_capital.head()

,Session Date,Client ID,Counselor,Session Type,Contact Type,Prep+Contact,Total Hours,Reporting Date,Client ID,Funding Type,Completion Status,Amount Approved
0,2013-01-02,L3155,"Zimmerman, Zack",Administrative,eCenter,0.25,0.25,None,None,None,None,NaN
1,2013-01-02,L3221,"Zimmerman, Zack",Administrative,eCenter,0.25,0.25,None,None,None,None,NaN
2,2013-01-02,L3222,"Zimmerman, Zack",Administrative,eCenter,0.25,0.25,None,None,None,None,NaN
3,2013-01-02,N1169,"Tuller, Jason",Initial/New,Center Site (face-to-face),2.50,2.50,None,None,None,None,NaN
4,2013-01-02,O8789,"Munoz, Luis",Administrative,eCenter,0.25,0.25,None,None,None,None,NaN


In [5]:
print(set(bus_capital['Completion Status']))
print(set(bus_capital['Funding Type']))

{'Withdrawn', 'Approved', 'In Progress', None}
{'COVID-19 Other SBA Disaster Loan', 'COVID-19 Paycheck Protection Program (SBA)', 'Certified Development Loan (CDC)', 'Community Advantage', 'Line of Credit', 'SEED Capital Fund', 'COVID-19 Shuttered Venue Operators (SVO) Grant', 'Community Express Loan', 'Private Loan', 'SBA Loan', None, 'Owner Investment', 'COVID-19 Economic Injury Disaster Loan (SBA)', 'COVID-19 Restaurant Revitalization Fund (RRF)', 'SBIR - Phase I w/STATE', 'Other Equity Investment', 'Micro-Loan', 'OSBN Other Fund', 'Seller Financing', 'Other Federal Loan', 'Commercial (Bank) Loan', 'Venture Capital', 'Community Dev. Block Grant (CDBG)', 'Stock Investment', 'Pre-Seed Grant (Prototyping) w/STATE', 'OTHER (All Other Type Loans)'}


In [6]:
query = """
SELECT 
    "Client ID",
    COUNT("Reporting Date") AS total_sessions,
    SUM("Total Hours") AS total_hours
FROM bus_capital
WHERE "Session Date" <= "Reporting Date"
GROUP BY "Client ID"
"""

client_funding_summary = pds.sqldf(query, locals())

In [7]:
average_sessions_query = """
SELECT AVG(total_sessions) AS avg_sessions
FROM client_funding_summary
"""
average_hours_query = """
SELECT AVG(total_hours) AS avg_hours
FROM client_funding_summary
"""

avg_sessions = pds.sqldf(average_sessions_query, locals())
avg_hours = pds.sqldf(average_hours_query, locals())

print(f"Average sessions before funding: {avg_sessions['avg_sessions'][0]:.2f}")
print(f"Average hours before funding: {avg_hours['avg_hours'][0]:.2f}")

Average sessions before funding: 22.64
Average hours before funding: 39.94


## Business Established

In [8]:
established.head()

,Milestone Date,Client ID,Counselor,Type
0,2023-12-14,O11630,"Guinotte, Patrick",NewBus
1,2023-10-16,O11401,"Babalola, Ola",NewBus
2,2023-09-13,O11389,"Ndungutse, Patrick",NewBus
3,2023-08-18,O11360,"Babalola, Ola",NewBus
4,2023-08-01,O11351,"Raabe, Emma",NewBus


In [9]:
established['Milestone Date'] = pd.to_datetime(established['Milestone Date'], errors='coerce')

query = """
SELECT 
    hours."Client ID",
    COUNT(hours."Session Date") AS total_sessions,
    SUM(hours."Total Hours") AS total_hours
FROM hours
LEFT JOIN established
ON hours."Client ID" = established."Client ID"
WHERE hours."Session Date" <= established."Milestone Date"
GROUP BY hours."Client ID"
"""

client_milestone_summary = pds.sqldf(query, locals())

In [10]:
average_sessions_query = """
SELECT AVG(total_sessions) AS avg_sessions
FROM client_milestone_summary
"""
average_hours_query = """
SELECT AVG(total_hours) AS avg_hours
FROM client_milestone_summary
"""

avg_sessions = pds.sqldf(average_sessions_query, locals())
avg_hours = pds.sqldf(average_hours_query, locals())

print(f"Average sessions before milestone: {avg_sessions['avg_sessions'][0]:.2f}")
print(f"Average hours before milestone: {avg_hours['avg_hours'][0]:.2f}")

Average sessions before milestone: 11.66
Average hours before milestone: 8.36


In [11]:
hours.columns

Index(['Session Date', 'Client ID', 'Counselor', 'Session Type',
       'Contact Type', 'Prep+Contact', 'Total Hours'],
      dtype='object')

In [12]:
print(min(hours['Session Date']))
print(max(hours['Session Date']))

2013-01-02 00:00:00
2023-12-22 00:00:00
